#   RM 294: Optimization Project 3 – Non-Linear Programming

## Group members: 

| Group Member | UT EID |
| ----------- | ----------- |
| Manvi Goyal | mg65952 |
| Shreyansh Agrawal | sa55742        |
| Rianna Patel | rnp599        |
| Nevin Arimilli | na24887        |


In [1]:
# importing required libraries

import numpy as np
import pandas as pd
import gurobipy as gp
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn import linear_model

from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model

c:\Users\shrey\anaconda3\envs\ShreyanshConda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# reading the files
train_df= pd.read_csv('training_data.csv')
test_df = pd.read_csv('test_data.csv')

# adding a column for intercept
train_df.insert(1,column='X0',value=np.ones(len(train_df)))
test_df.insert(1,column='X0',value=np.ones(len(test_df)))

# defining feature space and target variable
X_train = np.array(train_df.iloc[0:,1:])
Y_train = np.array(train_df['y'])
X_test = np.array(test_df.iloc[0:,1:])
Y_test = np.array(test_df['y'])

In [3]:
TIME_LIMIT  = 432 # time limit per iteration
M = 100
n_folds = 10

#defining th evalues of k for which we want to run the folds
k_list=np.linspace(5,50,10)

### We know that general form of the Quadratic Programming is:

Objective Function: $min_x$ $x^{T}$Qx+ $c^{T}$x

Constraints:
1. Ax$\leq$b
2. x$\geq$0

#### In our case, below are values for implementing MIQP:

Total Number of Decision Variables: *2m+1*

   1. m+1 beta values -  $\beta_0$, $\beta_1$,...,$\beta_m$
   2. m binary variables - $z_1$,$z_2$,...,$z_m$
   
##### Objective function: 
$min_{\beta,z}$ $\beta^{T}$($X^{T}$X)$\beta$+ (-2$y^{T}$X)$\beta$
    
On comparing with Quadratic form, we know:

   Q = $X^{T}$X and c = -2$y^{T}$X
   
Total number of Constraints = *2m+1*

   1. $\sum_{i=1}^{m} z_i$ $\leq$ k --> 1 constraint
   2. -$z_i$M $\leq$ $\beta_i$ *where i = 1,2,....,m* --> m constraints
   3. $\beta_i$ $\geq$ $z_i$M  *where i = 1,2,....,m* --> m constraints


### MIQP

In [4]:
# outputs the n folds for cross validation
def gen_test_val_split(X_train, n_folds):
    
    n_elements = int(X_train.shape[0]/n_folds)
    cv_list = []

    a = np.arange(0,X_train.shape[0])
    
    for i in range(0,n_folds):
        cv = np.random.choice(a,size=n_elements,replace=False)
        cv_list.append(cv)
        a = a[~np.isin(a,cv)]

    return cv_list  

In [5]:
# this fucntion is used to optimise the miqp model based on the training data passed and returns the optimization model from which we can then extract the beta values 

def optimize_miqp(X,y,k,TIME_LIMIT,M):
    
    # number of rows
    n = X.shape[0]
    # number of variables
    m = X.shape[1]-1

    # Quadratic part Q
    obj_quad= np.zeros((2*m+1, 2*m+1))
    obj_quad[0:m+1,0:m+1] = X.T @ X
        
    # Linear Part C
    obj_lin = np.zeros(2*m+1)
    obj_lin[:(m+1)] = -2*y.T @ X
        
    # Defining the constraints

    # Creating the A matrix
    A = np.zeros((2*m+1, 2*m+1))
    sense = ['']*A.shape[0]
    b = np.zeros(2*m+1)
    
    # big M constraint: b_j <= Mz_j 
    np.fill_diagonal(A[:m, 1:m+1], 1) 
    np.fill_diagonal(A[:m, m+1:2*m+1], -M)  

    # big M constraint: -Mz_j <= b_j
    np.fill_diagonal(A[m:-1, 1:m+1], 1) 
    np.fill_diagonal(A[m:-1, m+1:2*m+1], M)

    # Sum of the number of independent betas should be equal to k
    A[-1, m+1:] = 1
    sense = np.array(['<']*m + ['>']*m + ['<'])
    lb = np.array([np.NINF]+[-M]*m+[np.NINF]*m)

    b = np.concatenate((np.zeros(2*m), [k]))

    opt_model = gp.Model()
    opt_model_x = opt_model.addMVar(len(obj_quad),vtype=['C']*(m+1)+['B']*m, lb=lb) 
    opt_model_con = opt_model.addMConstrs(A, opt_model_x, sense, b)
    opt_model.setMObjective(obj_quad, obj_lin, 0, sense=gp.GRB.MINIMIZE)
    opt_model.Params.OutputFlag = 0
    opt_model.Params.TimeLimit = TIME_LIMIT
    opt_model.optimize()

    return opt_model


In [ ]:
# running 10-fold cross validation for different values of k

X = np.array(train_df.iloc[0:,1:])
y = np.array(train_df['y'])

cv_index = gen_test_val_split(X_train, n_folds)
a = np.arange(0,train_df.shape[1])

# list to store mse and beta values for each iteration
mse_list = []
beta_list = []

# dataframe to store the outputs of all th eietrations
col_list = ['k','n_fold']
col_list.extend(train_df.columns[1:])
col_list.extend(['mse'])
df_history = pd.DataFrame(columns=col_list)
counter = 0

for k in k_list:

    for i in range(0,n_folds):
        
        X_val = X[cv_index[i]]
        y_val = y[cv_index[i]]
        X_train = X[a[~np.isin(a,cv_index[i])]]
        y_train = y[a[~np.isin(a,cv_index[i])]]

        model = optimize_miqp(X=X_train,y=y_train,k=k,TIME_LIMIT=TIME_LIMIT,M=M)

        m=X_train.shape[1]-1
        beta = np.array(model.X)[0:m+1]
        beta_list.append(beta)

        y_pred = X_val@beta
        mse = mean_squared_error(y_pred, y_val)
        mse_list.append(mse)

        row_arr = [k,i]
        row_arr.extend(beta)
        row_arr.extend([mse])

        # storing to df
        df_history.loc[counter] = row_arr

        counter = counter+1

# storing the outputs in a csv file for reproducibility
df_history.to_csv("miqp_outputs.csv")

In [ ]:
# determining the best k using miqp
miqp_outputs = pd.read_csv("miqp_outputs.csv")

# note for the graph we have shown the total mse of all the folds - but i shouldn't matter if we show total mse or average mse since for each k we have 10 folds
miqp_outputs[['k','mse']].groupby('k').sum().sort_values(ascending=True,by='mse').plot(kind="bar")

In [ ]:
miqp_outputs[['k','mse']].groupby('k').sum().sort_values(ascending=True,by='mse')

From the above graph we can see that we get the lower cross validation error at k=10

In [9]:
# defining feature space and target variable
X_train = np.array(train_df.iloc[0:,1:])
Y_train = np.array(train_df['y'])
X_test = np.array(test_df.iloc[0:,1:])
Y_test = np.array(test_df['y'])

In [10]:
# now retraining the model using the complete training data with k=10
final_model = optimize_miqp(X=X_train,y=Y_train,k=10,TIME_LIMIT=6000,M=100)

m=X_train.shape[1]-1

# selcting the value of coefficients
beta = np.array(final_model.X)[0:m+1]

# predicting on test data
y_pred = X_test@beta

#mse = np.power(y_pred-Y_test,2).sum()
print("The test error is: "+ str(round(mean_squared_error(Y_test, y_pred),5)))

C:\Users\shrey\AppData\Local\Temp\ipykernel_28712\112114350.py:2: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  final_model = optimize_miqp(X=X_train,y=Y_train,k=10,TIME_LIMIT=6000,M=100)


The test error is: 2.33654


In [11]:
# for report purposes
df_beta = pd.DataFrame(beta)
df_beta.to_csv("miqp_beta_10.csv")

df_y_pred = pd.DataFrame(y_pred)
df_y_pred.to_csv("miqp_y__pred_10.csv")

In [12]:
y_pred = X_train@beta
print("train error is: "+ str(round(mean_squared_error(Y_train, y_pred),5)))

train error is: 2.39199


### Part 4

Use scikit learn to do 10-fold cross validation on the training set to pick lambda. Once you find 
the best value of lambda, fit a LASSO model to the entire training set using that value of lambda. 
With the betas you find in that LASSO model make a prediction of the y values in the test 
set.

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
train_df= pd.read_csv('training_data.csv')
test_df = pd.read_csv('test_data.csv')

In [15]:
# defining feature space and target variable
X_train = np.array(train_df.iloc[0:,1:])
Y_train = np.array(train_df['y'])
X_test = np.array(test_df.iloc[0:,1:])
Y_test = np.array(test_df['y'])

In [16]:
# standardizing the dataset
from sklearn.preprocessing import StandardScaler

# fitting on x train
std_trans = StandardScaler()
std_trans.fit(X_train)

#transforming on X_train and X_test
X_train_std = std_trans.transform(X_train)
X_test_std = std_trans.transform(X_test)


In [17]:

# fitting the lasso model to get the optimal lambda 
lasso_regression_cv = linear_model.LassoCV(cv=10).fit(X_train_std, Y_train)
print('Optimal Lambda is :', lasso_regression_cv.alpha_)

# calculating number of features
print('Total non-zero features using Lasso:', (lasso_regression_cv.coef_ != 0).sum())

#predicting value of y on the test data
y_pred=lasso_regression_cv.predict(X_test_std)

#calculating MSE on test data
print('The value of MSE on the test data is:', round(mean_squared_error(Y_test, y_pred),5))

#predicting value of y on the train data
y_pred=lasso_regression_cv.predict(X_train_std)

#calculating MSE on train data
print('The value of MSE on the train data is:', round(mean_squared_error(Y_train, y_pred),5))

Optimal Lambda is : 0.08471942409934509
Total non-zero features using Lasso: 18
The value of MSE on the test data is: 2.35667
The value of MSE on the train data is: 2.38644


For report purposes

In [18]:
#extracting the beta values for lasso
beta_lasso=lasso_regression_cv.coef_
df_beta_lasso = pd.DataFrame(beta_lasso)

# for report purposes
df_beta_lasso.to_csv("miqp_beta_lasso.csv")

In [19]:
# intercept from lasso
intercept_lasso=lasso_regression_cv.intercept_
intercept_lasso

1.2762324862184158

In [20]:
# for report purposes
#predicting value of y on the test data
y_pred=lasso_regression_cv.predict(X_test_std)
df_y_pred_lasso = pd.DataFrame(y_pred)
df_y_pred_lasso.to_csv("y_pred_lasso.csv")

Please note that all the charts and comparisons has been done in the Excel which is attached along with the submission.